In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
import pickle
from sklearn.externals import joblib

In [3]:
batch_size = 128

In [4]:
x = tf.placeholder(
    shape=[batch_size, 40, 64],
    dtype=tf.float32
)
y = tf.placeholder(
    shape=[batch_size, 4],
    dtype=tf.float32,
)

In [8]:
rnn_cell = tf.contrib.rnn.BasicRNNCell(
    num_units=128, 
    activation=tf.nn.tanh
)

In [9]:
x

<tf.Tensor 'Placeholder:0' shape=(128, 40, 64) dtype=float32>

In [10]:
outputs, last_states = tf.nn.dynamic_rnn(
    cell=rnn_cell,
    inputs=x,
    dtype=tf.float32,
)

In [11]:
outputs

<tf.Tensor 'rnn/transpose:0' shape=(128, 40, 128) dtype=float32>

In [12]:
last_states

<tf.Tensor 'rnn/while/Exit_2:0' shape=(128, 128) dtype=float32>

In [33]:
# fw_lstm_cell = tf.contrib.rnn.LSTMCell(
#     num_units=128, 
# )
# bw_lstm_cell = tf.contrib.rnn.LSTMCell(
#     num_units=128, 
# )
# outputs, final_fw, final_bw = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(
#     [fw_lstm_cell],
#     [bw_lstm_cell],
#     x,
#     dtype=tf.float32
# )

In [13]:
# final_fw =  tf.concat( [final_fw[0][0],final_fw[0][1]],axis=1)
# final_bw =  tf.concat( [final_bw[0][0],final_bw[0][1]],axis=1)
# final = tf.concat( [final_fw, final_bw], axis=1)

In [14]:
# last_states = tf.concat( [last_states[0], last_states[1]], axis=1)

In [15]:
output = tf.layers.Dense(units=4, activation=None)(last_states)
# output = tf.layers.Dense(units=4, activation=None)(final)

In [16]:
softmax = tf.nn.softmax(output)
print(softmax)

Tensor("Softmax:0", shape=(128, 4), dtype=float32)


In [17]:
out_loss = tf.nn.softmax_cross_entropy_with_logits(
    logits=output,
    labels=y
)

In [18]:
loss = tf.reduce_mean(out_loss)

In [20]:
loss

<tf.Tensor 'Mean:0' shape=() dtype=float32>

In [21]:
optimizer = tf.train.AdamOptimizer(
    learning_rate=0.0001
)

In [22]:
train_op = optimizer.minimize(loss)

In [18]:
# Слова в индексы эмбеддинга

In [26]:
import gensim
word2vec = gensim.models.Word2Vec.load('./../topic7-word2vec/word2vec32')

In [27]:
# word2vec.wv.syn0.shape

In [28]:
word_dictionary = {w:i for (i,w) in enumerate(word2vec.wv.index2word)}

In [29]:
def iter_data(batch_size=batch_size, is_train=True):
    max_length = 40
    batch_x, batch_y = [], []
    if is_train:
        path = './../topic7-word2vec/normalized_train.csv' 
    else:
        path = './../topic7-word2vec/normalized_test.csv'
    with open(path, 'r') as f:
        for line in f:
            if is_train:
                text, label = line.strip().split(',')
            else:
                text = line.strip()
            indexes = [
                word_dictionary.get(word) 
                for word in text.split() 
                if word in word_dictionary
            ]
            if not indexes:
                word_vectors = np.zeros( (max_length, 64), dtype=np.float32)
            else:
                word_vectors = word2vec.wv.syn0[indexes]

            if word_vectors.shape[0]< max_length:
                extra = np.array([
                    [0 for i in range(64)]
                    for j in range(max_length-word_vectors.shape[0])
                ])
                word_vectors = np.vstack(
                    [
                        word_vectors, extra
                    ]
                )

            if word_vectors.shape[0]> max_length:
                word_vectors = word_vectors[:max_length]
                
            batch_x.append(
                word_vectors
            )
            
            if is_train:
                _label_vec = np.zeros(4)
                _label_vec[int(label)] = 1
                batch_y.append(_label_vec)
            
            if len(batch_x) >=batch_size:
                yield np.array(batch_x), np.array(batch_y)
                batch_y, batch_x = [],[]
                
    if is_train==False:

        while len(batch_x)<batch_size:
            batch_x.append(
                np.zeros( (max_length, 64), dtype=np.float32)
            )
            
        yield np.array(batch_x), np.array(batch_y)

In [30]:
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    
    try:
        for epoch in range(10):
            for _tick, (_x,_y) in enumerate(iter_data()):

                _, _loss,_softmax = sess.run(
                    [train_op, loss, softmax],
                    feed_dict={
                        x: _x,
                        y: _y
                    }
                )

                if not _tick % 100:
                    print(epoch,_tick, _loss)

                    saver.save(sess=sess, save_path='./rnn.model')

    except KeyboardInterrupt:
        saver.save(sess=sess, save_path='./rnn.model')
        raise KeyboardInterrupt

0 0 1.45885
0 100 1.039
0 200 0.891637
0 300 0.717351
0 400 0.769657
0 500 0.762388
0 600 0.73753
0 700 0.58627
0 800 0.768123
0 900 0.531139
0 1000 0.639696
0 1100 0.550685
0 1200 0.557653
1 0 0.713187
1 100 0.66587
1 200 0.517392
1 300 0.485721
1 400 0.479181
1 500 0.41173
1 600 0.412269
1 700 0.286862
1 800 0.329401
1 900 0.260743
1 1000 0.369075
1 1100 0.256139
1 1200 0.26721
2 0 0.359015
2 100 0.414654
2 200 0.211947
2 300 0.302868
2 400 0.322763
2 500 0.276405
2 600 0.344258
2 700 0.236742
2 800 0.225065
2 900 0.24767
2 1000 0.297133
2 1100 0.216627
2 1200 0.278457
3 0 0.272494
3 100 0.352289
3 200 0.190759
3 300 0.289807
3 400 0.284497
3 500 0.258116
3 600 0.345632
3 700 0.222088
3 800 0.20313
3 900 0.230294
3 1000 0.258569
3 1100 0.188342
3 1200 0.279722
4 0 0.253748
4 100 0.334386
4 200 0.189797
4 300 0.283581
4 400 0.266978
4 500 0.239844
4 600 0.336768
4 700 0.203605
4 800 0.189082
4 900 0.235082
4 1000 0.22834
4 1100 0.175602
4 1200 0.276545
5 0 0.235321
5 100 0.32741
5 200

KeyboardInterrupt: 

In [31]:
with tf.Session() as sess:
    saver = tf.train.Saver()
    saver.restore(sess, save_path='./rnn.model')
    
    Y_probs = []
    
    for _tick, (_x,_y) in enumerate(iter_data(is_train=False)):

        _softmax = sess.run(
            [softmax],
            feed_dict={
                x: _x,
            }
        )
        for probs in _softmax[0]:
            Y_probs.append(probs)

INFO:tensorflow:Restoring parameters from ./rnn.model


In [32]:
Y_probs = np.array(Y_probs)

In [33]:
Y_probs.shape

(156672, 4)

In [34]:

with open('rnn_.csv','w') as sol:
    with open('../dz4/source_task_test_without_labels.csv', 'r') as f:
        sol.write('_id,label\n')
        f.readline()
        for (line, label) in zip(f, Y_probs):
            data = line.split(',')
            sol.write('%s,%s\n' % (data[0], label.argmax()))
#             yield preprocessing(line.strip().split(',')[1].lower()).split()

In [35]:
!open .